In [ ]:
# importing necessary libraries
import numpy as np 
import pandas as pd 
import seaborn as sns
import os

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

from nltk.corpus import stopwords
from nltk.util import ngrams

from wordcloud import WordCloud

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
df = pd.read_csv('/content/Data/ResultFile_2.csv')
myCopy = df.copy()
df.shape

(8245, 12)

In [ ]:
df.drop(['tweet_id'],axis = 1, inplace=True)

In [ ]:
from nltk.stem.porter import *
plt.style.use('seaborn')
import plotly.express as px
from plotly import graph_objs as go
from collections import Counter
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
for index, row in df.iterrows():
    tweet = row.text
    tweet = re.sub(r"https\S+", "", tweet) #remove links
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet) #replace #word with word
    
    df['text'].iloc[index:index+1] = tweet

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
#displaying tweets after above cleaning
df['text'].head()

0    Domestic Violence Awareness Hasn’t Caught Up W...
1                               Mother Nature’s MeToo 
2    It is 'high time' MeToo named and shamed men i...
3    “There's this idea that persistence involves s...
4                Meredith’s MeToo moment on TheOffice 
Name: text, dtype: object

In [ ]:
# Helper function to remove unwanted patterns
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

# Remove Twitter handles from the data 
df['text'] = np.vectorize(remove_pattern)(df['text'], "@[\w]*")


# Remove all words below 2 characters
df['text'] = df['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))

# Remove punctuations, numbers, and special characters
df['text'] = df['text'].str.replace("[^a-zA-Z#]", " ")

# Tokenize the tweets
tokenized_tweet = df['text'].apply(lambda x: x.split())
df['text_tokenised'] = tokenized_tweet
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])
df['text'] = tokenized_tweet



In [ ]:
df['text_tokenised']

0       [Domestic, Violence, Awareness, Hasn, t, Caugh...
1                              [Mother, Nature, s, MeToo]
2       [high, time, MeToo, named, and, shamed, men, m...
3       [There, s, this, idea, that, persistence, invo...
4                 [Meredith, s, MeToo, moment, TheOffice]
                              ...                        
8240      [Metoo, campaign, and, what, labour, laws, say]
8241      [Metoo, campaign, and, what, labour, laws, say]
8242    [Insightful, peace, Hala, Al, Karib, the, dang...
8243    [Olivia, Munn, Opens, About, Predator, Scandal...
8244    [Megan, Fox, feels, excluded, from, the, MeToo...
Name: text_tokenised, Length: 8245, dtype: object

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
from nltk import pos_tag
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string
stops = set(stopwords.words('english'))

stops

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [ ]:
def clean_review(words):
    output_words = []
    for w in words:
        if w.lower() not in stops:
            pos = pos_tag([w])
            clean_word = lemmatizer.lemmatize(w, pos = get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
documents = [clean_review(document) for document in df['text_tokenised']]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
documents[10]

['nsw',
 'police',
 'officer',
 'mitchell',
 'willey',
 'could',
 'kick',
 'force',
 'admit',
 'groped',
 'woman',
 'defqon',
 'dan']

In [ ]:
documents_corrected = [" ".join(x) for x in documents]

In [ ]:
import gensim
from gensim.models.word2vec import Word2Vec # the word2vec model gensim class
LabeledSentence = gensim.models.doc2vec.LabeledSentence # we'll talk about this down below


In [ ]:

model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34) 

model_w2v.train(tokenized_tweet, total_examples= len(df['text']), epochs=20)


(2753679, 12481260)

In [ ]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

In [ ]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 200)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



(8245, 200)

In [ ]:
wordvec_arrays[0]

array([-2.87751252e-02, -1.27613247e-01, -3.95850575e-02, -4.48977212e-02,
        8.95629269e-02,  7.86018535e-02,  5.54860917e-02,  5.24534558e-02,
       -1.51761148e-01, -4.49146405e-02, -1.12600913e-01,  2.36430398e-02,
       -6.74020607e-02,  3.06507831e-02, -4.82180082e-02,  2.66598855e-02,
       -4.56411819e-02,  9.25875319e-02,  1.01780203e-01, -4.15861461e-02,
        2.80758128e-02,  7.35673078e-02, -1.35074248e-02, -7.85657527e-02,
        1.92130656e-02,  5.41105454e-02, -7.71754864e-02,  2.86627109e-02,
       -4.09461424e-02,  1.14541751e-02,  7.20762395e-02,  6.17711605e-02,
        3.37541003e-02,  7.22711550e-03, -6.10903416e-02,  1.71220853e-01,
        7.42829351e-02, -1.42567324e-01,  2.83183605e-03, -1.33439746e-01,
        2.12685499e-02, -5.08787523e-02, -4.77514191e-02, -2.87909063e-02,
       -6.43878638e-02, -1.20233184e-01, -5.86988906e-02, -1.58671903e-03,
        2.76912043e-03, -3.91269456e-02,  4.92277628e-02,  5.81662488e-03,
       -9.92105039e-02, -

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
from sklearn.model_selection import train_test_split  
## splitting for relevance
X_train_relevance, X_test_relevance, y_train_relevance, y_test_relevance = train_test_split(wordvec_df,df['relevance'] , test_size=0.3, random_state=0)

## splitting for directed_hate
X_train_dh, X_test_dh, y_train_dh, y_test_dh = train_test_split(wordvec_df,df['directed_hate'] , test_size=0.3, random_state=0)

## splitting for generalised_hate
X_train_gh, X_test_gh, y_train_gh, y_test_gh = train_test_split(wordvec_df,df['generalised_hate'] , test_size=0.3, random_state=0)

## splitting for sarcasm
X_train_sarcasm, X_test_sarcasm, y_train_sarcasm, y_test_sarcasm = train_test_split(wordvec_df,df['sarcasm'] , test_size=0.3, random_state=0)

## splitting for allegation
X_train_allegation, X_test_allegation, y_train_allegation, y_test_allegation = train_test_split(wordvec_df,df['allegation'] , test_size=0.3, random_state=0)

## splitting for justification
X_train_justification, X_test_justification, y_train_justification, y_test_justification = train_test_split(wordvec_df,df['justification'] , test_size=0.3, random_state=0)

## splitting for refutation
X_train_refutation, X_test_refutation, y_train_refutation, y_test_refutation = train_test_split(wordvec_df,df['refutation'] , test_size=0.3, random_state=0)

## splitting for Support
X_train_Support, X_test_Support, y_train_Support, y_test_Support = train_test_split(wordvec_df,df['Favour'] , test_size=0.3, random_state=0)

## splitting for oppose
X_train_oppose, X_test_oppose, y_train_oppose, y_test_oppose = train_test_split(wordvec_df,df['oppose'] , test_size=0.3, random_state=0)




In [ ]:
##training for relevance
from sklearn.ensemble import RandomForestClassifier
text_classifier = RandomForestClassifier(n_estimators=100, random_state=0)  
text_classifier.fit(X_train_relevance, y_train_relevance)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
predictions = text_classifier.predict(X_test_relevance)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test_relevance,predictions))  
print(classification_report(y_test_relevance,predictions))  
print(accuracy_score(y_test_relevance, predictions))

[[  38  599]
 [  79 1758]]
              precision    recall  f1-score   support

           0       0.32      0.06      0.10       637
           1       0.75      0.96      0.84      1837

    accuracy                           0.73      2474
   macro avg       0.54      0.51      0.47      2474
weighted avg       0.64      0.73      0.65      2474

0.7259498787388844


In [ ]:
##training for directed_hate
text_classifier.fit(X_train_dh, y_train_dh)
predictions_dh = text_classifier.predict(X_test_dh)
print(confusion_matrix(y_test_dh,predictions_dh))  
print(classification_report(y_test_dh,predictions_dh))  
print(accuracy_score(y_test_dh, predictions_dh))

[[2356   12]
 [ 105    1]]
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      2368
           1       0.08      0.01      0.02       106

    accuracy                           0.95      2474
   macro avg       0.52      0.50      0.50      2474
weighted avg       0.92      0.95      0.93      2474

0.9527081649151172


In [ ]:
##training for generalised_hate
text_classifier.fit(X_train_gh, y_train_gh)
predictions_gh = text_classifier.predict(X_test_gh)
print(confusion_matrix(y_test_gh,predictions_gh))  
print(classification_report(y_test_gh,predictions_gh))  
print(accuracy_score(y_test_gh, predictions_gh))

[[2397   12]
 [  64    1]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2409
           1       0.08      0.02      0.03        65

    accuracy                           0.97      2474
   macro avg       0.53      0.51      0.51      2474
weighted avg       0.95      0.97      0.96      2474

0.9692805173807599


In [ ]:
##training for sarcasm
text_classifier.fit(X_train_sarcasm, y_train_sarcasm)
predictions_sarcasm = text_classifier.predict(X_test_sarcasm)
print(confusion_matrix(y_test_sarcasm,predictions_sarcasm))  
print(classification_report(y_test_sarcasm,predictions_sarcasm))  
print(accuracy_score(y_test_sarcasm, predictions_sarcasm))

[[2420    5]
 [  48    1]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      2425
           1       0.17      0.02      0.04        49

    accuracy                           0.98      2474
   macro avg       0.57      0.51      0.51      2474
weighted avg       0.96      0.98      0.97      2474

0.9785772029102667


In [ ]:
##training for allegation
text_classifier.fit(X_train_allegation, y_train_allegation)
predictions_allegation = text_classifier.predict(X_test_allegation)
print(confusion_matrix(y_test_allegation,predictions_allegation))  
print(classification_report(y_test_allegation,predictions_allegation))  
print(accuracy_score(y_test_allegation, predictions_allegation))

[[2310   19]
 [ 142    3]]
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      2329
           1       0.14      0.02      0.04       145

    accuracy                           0.93      2474
   macro avg       0.54      0.51      0.50      2474
weighted avg       0.89      0.93      0.91      2474

0.9349232012934519


In [ ]:
##training for justification
text_classifier.fit(X_train_justification, y_train_justification)
predictions_justification= text_classifier.predict(X_test_justification)
print(confusion_matrix(y_test_justification,predictions_justification))  
print(classification_report(y_test_justification,predictions_justification))  
print(accuracy_score(y_test_justification, predictions_justification))

[[2390    8]
 [  75    1]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.98      2398
           1       0.11      0.01      0.02        76

    accuracy                           0.97      2474
   macro avg       0.54      0.50      0.50      2474
weighted avg       0.94      0.97      0.95      2474

0.9664510913500404


In [ ]:
##training for refutation
text_classifier.fit(X_train_refutation, y_train_refutation)
predictions_refutation= text_classifier.predict(X_test_refutation)
print(confusion_matrix(y_test_refutation,predictions_refutation))  
print(classification_report(y_test_refutation,predictions_refutation))  
print(accuracy_score(y_test_refutation, predictions_refutation))

[[2409    2]
 [  62    1]]
              precision    recall  f1-score   support

           0       0.97      1.00      0.99      2411
           1       0.33      0.02      0.03        63

    accuracy                           0.97      2474
   macro avg       0.65      0.51      0.51      2474
weighted avg       0.96      0.97      0.96      2474

0.9741309620048505


In [ ]:
##training for Support
text_classifier.fit(X_train_Support, y_train_Support)
predictions_Support= text_classifier.predict(X_test_Support)
print(confusion_matrix(y_test_Support,predictions_Support))  
print(classification_report(y_test_Support,predictions_Support))  
print(accuracy_score(y_test_Support, predictions_Support))

[[1567   90]
 [ 782   35]]
              precision    recall  f1-score   support

           0       0.67      0.95      0.78      1657
           1       0.28      0.04      0.07       817

    accuracy                           0.65      2474
   macro avg       0.47      0.49      0.43      2474
weighted avg       0.54      0.65      0.55      2474

0.6475343573160873


In [ ]:
##training for oppose
text_classifier.fit(X_train_oppose, y_train_oppose)
predictions_oppose= text_classifier.predict(X_test_oppose)
print(confusion_matrix(y_test_oppose,predictions_oppose))  
print(classification_report(y_test_oppose,predictions_oppose))  
print(accuracy_score(y_test_oppose, predictions_oppose))

[[2274   17]
 [ 180    3]]
              precision    recall  f1-score   support

           0       0.93      0.99      0.96      2291
           1       0.15      0.02      0.03       183

    accuracy                           0.92      2474
   macro avg       0.54      0.50      0.49      2474
weighted avg       0.87      0.92      0.89      2474

0.9203718674211803


In [ ]:
##training for relevance
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

GB_model = GaussianNB()

GB_model.fit(X_train_relevance, y_train_relevance)
y_predict_relevance= GB_model.predict(X_test_relevance)
print(accuracy_score(y_test_relevance, y_predict_relevance))

0.30072756669361356


In [ ]:
##training for directed_hate
GB_model.fit(X_train_dh, y_train_dh)
y_predict_dh= GB_model.predict(X_test_dh)
print(accuracy_score(y_test_dh, y_predict_dh))

0.6548100242522231


In [ ]:
##training for generalised_hate
GB_model.fit(X_train_gh, y_train_gh)
y_predict_gh= GB_model.predict(X_test_gh)
print(accuracy_score(y_test_gh, y_predict_gh))

0.9256265157639451


In [ ]:
##training for sarcasm
GB_model.fit(X_train_sarcasm, y_train_sarcasm)
y_predict_sarcasm= GB_model.predict(X_test_sarcasm)
print(accuracy_score(y_test_sarcasm, y_predict_sarcasm))

0.936944219886823


In [ ]:
##training for allegation
GB_model.fit(X_train_allegation, y_train_allegation)
y_predict_allegation= GB_model.predict(X_test_allegation)
print(accuracy_score(y_test_allegation, y_predict_allegation))

0.9106709781729992


In [ ]:
##training for justification
GB_model.fit(X_train_justification, y_train_justification)
y_predict_justification= GB_model.predict(X_test_justification)
print(accuracy_score(y_test_justification, y_predict_justification))

0.896928051738076


In [ ]:
##training for refutation
GB_model.fit(X_train_refutation, y_train_refutation)
y_predict_refutation= GB_model.predict(X_test_refutation)
print(accuracy_score(y_test_refutation, y_predict_refutation))

0.8779304769603881


In [ ]:
##training for Support
GB_model.fit(X_train_Support, y_train_Support)
y_predict_Support= GB_model.predict(X_test_Support)
print(accuracy_score(y_test_Support, y_predict_Support))

0.5804365400161682


In [ ]:
##training for oppose
GB_model.fit(X_train_oppose, y_train_oppose)
y_predict_oppose= GB_model.predict(X_test_oppose)
print(accuracy_score(y_test_oppose, y_predict_oppose))

0.8423605497170574


In [ ]:
##training for relevance
from sklearn import svm
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_relevance, y_train_relevance)
rbf_pred_relevance = rbf.predict(X_test_relevance)
rbf_accuracy = accuracy_score(y_test_relevance, rbf_pred_relevance)
print(rbf_accuracy)

0.7425222312045271


In [ ]:
##training for directed_hate
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_dh, y_train_dh)
rbf_pred_dh = rbf.predict(X_test_dh)
rbf_accuracy = accuracy_score(y_test_dh, rbf_pred_dh)
print(rbf_accuracy)

0.9571544058205336


In [ ]:
##training for generalised_hate
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_gh, y_train_gh)
rbf_pred_gh = rbf.predict(X_test_gh)
rbf_accuracy = accuracy_score(y_test_gh, rbf_pred_gh)
print(rbf_accuracy)

0.9737267582861763


In [ ]:
##training for sarcasm
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_sarcasm, y_train_sarcasm)
rbf_pred_sarcasm = rbf.predict(X_test_sarcasm)
rbf_accuracy = accuracy_score(y_test_sarcasm, rbf_pred_sarcasm)
print(rbf_accuracy)

0.9801940177849636


In [ ]:
##training for allegation
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_allegation, y_train_allegation)
rbf_pred_allegation = rbf.predict(X_test_allegation)
rbf_accuracy = accuracy_score(y_test_allegation, rbf_pred_allegation)
print(rbf_accuracy)

0.9413904607922393


In [ ]:
##training for justification
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_justification, y_train_justification)
rbf_pred_justification = rbf.predict(X_test_justification)
rbf_accuracy = accuracy_score(y_test_justification, rbf_pred_justification)
print(rbf_accuracy)

0.9692805173807599


In [ ]:
##training for refutation
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_refutation, y_train_refutation)
rbf_pred_refutation = rbf.predict(X_test_refutation)
rbf_accuracy = accuracy_score(y_test_refutation, rbf_pred_refutation)
print(rbf_accuracy)

0.9745351657235246


In [ ]:
##training for Support
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_Support, y_train_Support)
rbf_pred_Support = rbf.predict(X_test_Support)
rbf_accuracy = accuracy_score(y_test_Support, rbf_pred_Support)
print(rbf_accuracy)

0.6697655618431689


In [ ]:
##training for Oppose
rbf = svm.SVC(kernel='rbf', gamma=0.5, C=0.1).fit(X_train_oppose, y_train_oppose)
rbf_pred_oppose = rbf.predict(X_test_oppose)
rbf_accuracy = accuracy_score(y_test_oppose, rbf_pred_oppose)
print(rbf_accuracy)

0.9260307194826193
